# Google Vertex AI Search | Google Vertex AI Search[Vertex AI Search](https://cloud.google.com/enterprise-search)（かつてはGenerative AI App BuilderにおけるEnterprise Searchとして知られていた）は、Google Cloudが提供する[Vertex AI](https://cloud.google.com/vertex-ai)機械学習プラットフォームの一部です。> [Vertex AI Search](https://cloud.google.com/enterprise-search) (formerly known as Enterprise Search on Generative AI App Builder) is a part of the [Vertex AI](https://cloud.google.com/vertex-ai) machine learning platform offered by Google Cloud.Vertex AI Searchは、企業が顧客や従業員向けに生成型AIを活用した検索エンジンを迅速に構築できるよう支援します。これは、セマンティック検索を含むGoogle Searchの技術に基づいており、自然言語処理と機械学習の技術を用いて、従来のキーワードベースの検索技術よりも関連性の高い結果を提供することができます。これにより、コンテンツ内の関係性とユーザーのクエリ入力からの意図を推測することが可能です。また、Vertex AI Searchは、ユーザーの検索方法とコンテンツの関連性を理解するGoogleの専門知識を活かし、表示される結果の順序付けにもその要素を取り入れています。> Vertex AI Search lets organizations quickly build generative AI powered search engines for customers and employees. It's underpinned by a variety of Google Search technologies, including semantic search, which helps deliver more relevant results than traditional keyword-based search techniques by using natural language processing and machine learning techniques to infer relationships within the content and intent from the user’s query input. Vertex AI Search also benefits from Google’s expertise in understanding how users search and factors in content relevance to order displayed results.Vertex AI Searchは、Google Cloud Consoleおよびエンタープライズワークフロー統合用のAPIを通じて利用可能です。> Vertex AI Search is available in the Google Cloud Console and via an API for enterprise workflow integration.このノートブックは、Vertex AI Searchを設定し、Vertex AI Search retrieverを使用する方法を示しています。Vertex AI Search retrieverは[Pythonクライアントライブラリ](https://cloud.google.com/generative-ai-app-builder/docs/libraries#client-libraries-install-python)を内包しており、それを利用して[Search Service API](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.services.search_service)にアクセスします。> This notebook demonstrates how to configure Vertex AI Search and use the Vertex AI Search retriever. The Vertex AI Search retriever encapsulates the [Python client library](https://cloud.google.com/generative-ai-app-builder/docs/libraries#client-libraries-install-python) and uses it to access the [Search Service API](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.services.search_service).

## Install pre-requisites | 前提条件となるソフトウェアをインストールしますVertex AI Search retrieverを使用するには、`google-cloud-discoveryengine`パッケージをインストールする必要があります。> You need to install the `google-cloud-discoveryengine` package to use the Vertex AI Search retriever.

In [ ]:
! pip install google-cloud-discoveryengine

## Configure access to Google Cloud and Vertex AI Search | Google CloudとVertex AI Searchへのアクセスを設定するVertex AI Searchは、2023年8月からホワイトリストなしで一般に利用可能となっています。> Vertex AI Search is generally available without allowlist as of August 2023.リトリーバーを使用する前に、以下のステップを完了する必要があります：> Before you can use the retriever, you need to complete the following steps:

### Create a search engine and populate an unstructured data store | 検索エンジンを作成し、非構造化データストアを充実させる* [Vertex AI Search Getting Started guide](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search)に記載されている指示に従って、Google CloudプロジェクトとVertex AI Searchを設定してください。  > Follow the instructions in the [Vertex AI Search Getting Started guide](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search) to set up a Google Cloud project and Vertex AI Search.* [Google Cloud Consoleを使用して非構造化データストアを作成する](https://cloud.google.com/generative-ai-app-builder/docs/create-engine-es#unstructured-data)  > [Use the Google Cloud Console to create an unstructured data store](https://cloud.google.com/generative-ai-app-builder/docs/create-engine-es#unstructured-data)  * `gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs` Cloud Storageフォルダにある例示用のPDFドキュメントで埋めてください。    > Populate it with the example PDF documents from the `gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs` Cloud Storage folder.  * `Cloud Storage (without metadata)`オプションを使用してください。    > Make sure to use the `Cloud Storage (without metadata)` option.

### Set credentials to access Vertex AI Search API | Vertex AI Search APIにアクセスするための認証情報を設定しますVertex AI Search retrieverが使用する[Vertex AI Searchクライアントライブラリ](https://cloud.google.com/generative-ai-app-builder/docs/libraries)は、プログラムによるGoogle Cloudへの認証をサポートする高レベルな言語機能を提供します。クライアントライブラリは[Application Default Credentials (ADC)](https://cloud.google.com/docs/authentication/application-default-credentials)をサポートしており、定義された一連の場所で認証情報を探し、それらの認証情報を使用してAPIへのリクエストを認証します。ADCを使用すると、アプリケーションコードを変更することなく、ローカル開発環境や本番環境など、さまざまな環境で認証情報をアプリケーションに利用可能にすることができます。> The [Vertex AI Search client libraries](https://cloud.google.com/generative-ai-app-builder/docs/libraries) used by the Vertex AI Search retriever provide high-level language support for authenticating to Google Cloud programmatically.> Client libraries support [Application Default Credentials (ADC)](https://cloud.google.com/docs/authentication/application-default-credentials); the libraries look for credentials in a set of defined locations and use those credentials to authenticate requests to the API.> With ADC, you can make credentials available to your application in a variety of environments, such as local development or production, without needing to modify your application code.[Google Colab](https://colab.research.google.com)で実行している場合は、`google.colab.auth.authenticate_user`で認証してください。それ以外の場合は、アプリケーションのデフォルトクレデンシャルが適切に設定されていることを確認するために、[サポートされている方法](https://cloud.google.com/docs/authentication/application-default-credentials)のいずれかをフォローしてください。> If running in [Google Colab](https://colab.google) authenticate with `google.colab.google.auth` otherwise follow one of the [supported methods](https://cloud.google.com/docs/authentication/application-default-credentials) to make sure that you Application Default Credentials are properly set.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

## Configure and use the Vertex AI Search retriever | Vertex AI Search リトリーバーの設定と使用方法Vertex AI Search retrieverは、`langchain.retriever.GoogleVertexAISearchRetriever` クラスで実装されています。`get_relevant_documents` メソッドは、各ドキュメントの `page_content` フィールドにドキュメントの内容が格納されている `langchain.schema.Document` ドキュメントのリストを返します。Vertex AI Searchで使用されるデータタイプ（ウェブサイト、構造化、非構造化）に応じて、`page_content` フィールドは以下のように格納されます：> The Vertex AI Search retriever is implemented in the `langchain.retriever.GoogleVertexAISearchRetriever` class. The `get_relevant_documents` method returns a list of `langchain.schema.Document` documents where the `page_content` field of each document is populated the document content.> Depending on the data type used in Vertex AI Search (website, structured or unstructured) the `page_content` field is populated as follows:* 高度なインデックスを持つウェブサイト：クエリに一致する`extractive answer`。`metadata`フィールドは、セグメントや回答が抽出されたドキュメントのメタデータ（存在する場合）で埋められます。  > Website with advanced indexing: an `extractive answer` that matches a query. The `metadata` field is populated with metadata (if any) of the document from which the segments or answers were extracted.* 非構造化データソース：クエリに一致する `extractive segment` または `extractive answer` のいずれかです。`metadata` フィールドには、セグメントや回答が抽出されたドキュメントのメタデータ（存在する場合）が記入されています。  > Unstructured data source: either an `extractive segment` or an `extractive answer` that matches a query. The `metadata` field is populated with metadata (if any) of the document from which the segments or answers were extracted.* 構造化データソース：構造化データソースから返される全てのフィールドを含むJSON形式の文字列です。`metadata`フィールドには、ドキュメントのメタデータ（存在する場合）が格納されています  > Structured data source: a string json containing all the fields returned from the structured data source. The `metadata` field is populated with metadata (if any) of the document### Extractive answers & extractive segments | 抽出型の回答と抽出型のセグメント抽出型回答とは、各検索結果に返される逐語的なテキストです。これは元のドキュメントから直接抽出されます。抽出型回答は通常、ウェブページの上部近くに表示され、エンドユーザーに彼らのクエリに文脈的に関連する簡潔な回答を提供するために使用されます。抽出型回答は、ウェブサイトや非構造化検索において利用可能です。> An extractive answer is verbatim text that is returned with each search result. It is extracted directly from the original document. Extractive answers are typically displayed near the top of web pages to provide an end user with a brief answer that is contextually relevant to their query. Extractive answers are available for website and unstructured search.抽出セグメントとは、各検索結果に返される逐語的なテキストのことです。抽出セグメントは通常、抽出回答よりも冗長です。抽出セグメントはクエリの回答として表示されることもあり、後処理タスクを実行するためや、大規模言語モデルが回答や新しいテキストを生成するための入力としても使用できます。抽出セグメントは非構造化検索で利用可能です。> An extractive segment is verbatim text that is returned with each search result. An extractive segment is usually more verbose than an extractive answer. Extractive segments can be displayed as an answer to a query, and can be used to perform post-processing tasks and as input for large language models to generate answers or new text. Extractive segments are available for unstructured search.抽出的なセグメントや抽出的な回答についての詳細は、[製品ドキュメント](https://cloud.google.com/generative-ai-app-builder/docs/snippets)を参照してください。> For more information about extractive segments and extractive answers refer to [product documentation](https://cloud.google.com/generative-ai-app-builder/docs/snippets).注意：抽出型セグメントを使用するには、[Enterprise edition](https://cloud.google.com/generative-ai-app-builder/docs/about-advanced-features#enterprise-features)の機能を有効にする必要があります。> NOTE: Extractive segments require the [Enterprise edition](https://cloud.google.com/generative-ai-app-builder/docs/about-advanced-features#enterprise-features) features to be enabled.リトリーバーのインスタンスを作成する際には、どのデータストアにアクセスするか、自然言語クエリがどのように処理されるかを制御するいくつかのパラメータを指定できます。これには、抽出型回答とセグメントの設定も含まれます。> When creating an instance of the retriever you can specify a number of parameters that control which data store to access and how a natural language query is processed, including configurations for extractive answers and segments.### The mandatory parameters are: | 必須パラメータは以下の通りです：* `project_id` - あなたのGoogle CloudプロジェクトID。  > `project_id` - Your Google Cloud Project ID.* `location_id` - データストアの場所。  > `location_id` - The location of the data store.  * `global`（デフォルト）    > `global` (default)  * `us`    > `us`  * eu    > `eu`* `data_store_id` - 使用したいデータストアのIDです。  > `data_store_id` - The ID of the data store you want to use.  * 注意：以前のバージョンのリトリバーでは、これは `search_engine_id` と呼ばれていました。    > Note: This was called `search_engine_id` in previous versions of the retriever.`project_id`と`data_store_id`パラメータは、リトリーバのコンストラクタで明示的に指定することも、環境変数`PROJECT_ID`と`DATA_STORE_ID`を通じて指定することもできます。> The `project_id` and `data_store_id` parameters can be provided explicitly in the retriever's constructor or through the environment variables - `PROJECT_ID` and `DATA_STORE_ID`.また、以下を含むいくつかのオプションパラメーターを設定することもできます：> You can also configure a number of optional parameters, including:* `max_documents` - 抽出的なセグメントや抽出的な回答を提供するために使用される文書の最大数  > `max_documents` - The maximum number of documents used to provide extractive segments or extractive answers* `get_extractive_answers` - デフォルトでは、リトリバーは抽出的なセグメントを返すように設定されています。  > `get_extractive_answers` - By default, the retriever is configured to return extractive segments.  * `engine_data_type`が`0`（非構造化）に設定されている場合にのみ使用され、このフィールドを`True`に設定すると抽出型の回答が返されます。    > Set this field to `True` to return extractive answers. This is used only when `engine_data_type` set to `0` (unstructured)* `max_extractive_answer_count` - 各検索結果で返される抽出型回答の最大数。  > `max_extractive_answer_count` - The maximum number of extractive answers returned in each search result.  * 最大で5つの回答が返されます。これは`engine_data_type`が`0`（非構造化）に設定されている場合にのみ使用されます。    > At most 5 answers will be returned. This is used only when `engine_data_type` set to `0` (unstructured).* `max_extractive_segment_count` - 各検索結果で返される抽出セグメントの最大数。  > `max_extractive_segment_count` - The maximum number of extractive segments returned in each search result.  * 現在、1つのセグメントが返されます。これは`engine_data_type`が`0`（非構造化）に設定されている場合にのみ使用されます。    > Currently one segment will be returned. This is used only when `engine_data_type` set to `0` (unstructured).* `filter` - データストア内のドキュメントに関連付けられたメタデータに基づいた検索結果のフィルター式です。  > `filter` - The filter expression for the search results based on the metadata associated with the documents in the data store.* `query_expansion_condition` - クエリ拡張が行われるべき条件を決定する仕様。  > `query_expansion_condition` - Specification to determine under which conditions query expansion should occur.  * `0` - クエリ拡張条件が指定されていません。この場合、サーバーの動作はデフォルトで無効に設定されています。    > `0` - Unspecified query expansion condition. In this case, server behavior defaults to disabled.  * `1` - クエリ拡張を無効にしました。SearchResponse.total\_sizeがゼロでも、検索クエリは厳密にそのまま使用されます。    > `1` - Disabled query expansion. Only the exact search query is used, even if SearchResponse.total\_size is zero.  * `2` - Search APIによって構築された自動クエリ拡張。    > `2` - Automatic query expansion built by the Search API.* `engine_data_type` - Vertex AI Searchのデータタイプを定義します  > `engine_data_type` - Defines the Vertex AI Search data type  * `0` - 構造化されていないデータ    > `0` - Unstructured data  * `1` - 構造化データ    > `1` - Structured data  * `2` - [Advanced Website Indexing](https://cloud.google.com/generative-ai-app-builder/docs/about-advanced-features#advanced-website-indexing)を使用したウェブサイトのデータ    > `2` - Website data with [Advanced Website Indexing](https://cloud.google.com/generative-ai-app-builder/docs/about-advanced-features#advanced-website-indexing)### Migration guide for `GoogleCloudEnterpriseSearchRetriever` | `GoogleCloudEnterpriseSearchRetriever`の移行ガイド以前のバージョンでは、このリトリーバーは `GoogleCloudEnterpriseSearchRetriever` と呼ばれていました。製品の挙動に変更があったため、一般公開の開始後にリトリーバーに対して後方互換性のない変更を行う必要が生じました。> In previous versions, this retriever was called `GoogleCloudEnterpriseSearchRetriever`. Some backwards-incompatible changes had to be made to the retriever after the General Availability launch due to changes in the product behavior.新しいリトリーバーに更新するには、以下の変更を行ってください：> To update to the new retriever, make the following changes:* `from langchain.retrievers import GoogleCloudEnterpriseSearchRetriever` から `from langchain.retrievers import GoogleVertexAISearchRetriever` へのインポートを変更してください。  > Change the import from: `from langchain.retrievers import GoogleCloudEnterpriseSearchRetriever` -> `from langchain.retrievers import GoogleVertexAISearchRetriever`.* `GoogleCloudEnterpriseSearchRetriever`から`GoogleVertexAISearchRetriever`へのすべてのクラス参照を変更してください。  > Change all class references from `GoogleCloudEnterpriseSearchRetriever` -> `GoogleVertexAISearchRetriever`.* クラスの初期化時に、`search_engine_id` パラメータ名を `data_store_id` に変更してください。  > Upon class initialization, change the `search_engine_id` parameter name to `data_store_id`.

### Configure and use the retriever for **unstructured** data with extractive segments | 抽出的なセグメントを持つ**非構造化**データ用のリトリーバーを設定し、使用する

In [ ]:
from langchain.retrievers import (
    GoogleVertexAIMultiTurnSearchRetriever,
    GoogleVertexAISearchRetriever,
)

PROJECT_ID = "<YOUR PROJECT ID>"  # Set to your Project ID
LOCATION_ID = "<YOUR LOCATION>"  # Set to your data store location
DATA_STORE_ID = "<YOUR DATA STORE ID>"  # Set to your data store ID

In [ ]:
retriever = GoogleVertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION_ID,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
)

In [ ]:
query = "What are Alphabet's Other Bets?"

result = retriever.get_relevant_documents(query)
for doc in result:
    print(doc)

### Configure and use the retriever for **unstructured** data with extractive answers | 抽出的な回答を持つ**非構造化**データ用のリトリーバーの設定と使用方法

In [ ]:
retriever = GoogleVertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION_ID,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
    max_extractive_answer_count=3,
    get_extractive_answers=True,
)

result = retriever.get_relevant_documents(query)
for doc in result:
    print(doc)

### Configure and use the retriever for **structured** data | 構造化されたデータ用のリトリーバーを設定して使用する

In [ ]:
retriever = GoogleVertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION_ID,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
    engine_data_type=1,
)

result = retriever.get_relevant_documents(query)
for doc in result:
    print(doc)

### Configure and use the retriever for **website** data with Advanced Website Indexing | Advanced Website Indexingを用いて**ウェブサイト**データのリトリーバーを設定し、使用する方法

In [ ]:
retriever = GoogleVertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION_ID,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
    max_extractive_answer_count=3,
    get_extractive_answers=True,
    engine_data_type=2,
)

result = retriever.get_relevant_documents(query)
for doc in result:
    print(doc)

### Configure and use the retriever for multi-turn search | マルチターン検索のためのリトリーバーを設定して使用する[フォローアップ付き検索](https://cloud.google.com/generative-ai-app-builder/docs/multi-turn-search)は、生成AIモデルに基づいており、通常の非構造化データ検索とは異なります。> [Search with follow-ups](https://cloud.google.com/generative-ai-app-builder/docs/multi-turn-search) is based on generative AI models and it is different from the regular unstructured data search.

In [ ]:
retriever = GoogleVertexAIMultiTurnSearchRetriever(
    project_id=PROJECT_ID, location_id=LOCATION_ID, data_store_id=DATA_STORE_ID
)

result = retriever.get_relevant_documents(query)
for doc in result:
    print(doc)